# Programação Orientada a Objetos (POO)
## Tema 7 – Bases de Dados Relacionais

### Parte II – Acesso a BD MySQL com Python

Jaime A. Martins

(CEOT/ISE/UAlg - jamartins@ualg.pt)

###### Autores: Jaime Martins [v2]; Pedro Cardoso [v1]

## Conexão à base de dados usando um `connector`

O método `connect()` do módulo `mysql` cria uma conexão a um servidor MySQL e devolve um objeto do tipo `MySQLConnection`.
   * ver https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html para outros argumentos e opções

In [1]:
import mysql.connector

conn = mysql.connector.connect(
    user="sensors",  # make sure you have a user:sensors/pw:##sensors## with access rights to the sensors database
    password="##sensors##",
    host="localhost",  # replace 'localhost', if necessary
    database="sensors",
)
conn

Algumas informações sobre a conexão podem ser consultadas no `__dict__`

In [2]:
conn.__dict__

{'_cmysql': <_mysql_connector.MySQL at 0x1c1ece889c0>,
 '_columns': [],
 '_plugin_dir': 'c:\\Users\\virtu\\miniforge3\\envs\\aulas\\Lib\\site-packages\\mysql\\vendor\\plugin',
 'converter': None,
 '_client_flags': 1286669,
 '_charset_id': 45,
 '_sql_mode': None,
 '_time_zone': None,
 '_autocommit': False,
 '_server_version': (5, 5, 5),
 '_handshake': {'protocol': 10,
  'server_version_original': '5.5.5-10.4.27-MariaDB',
  'server_threadid': 111,
  'charset': None,
  'server_status': None,
  'auth_plugin': None,
  'auth_data': None,
  'capabilities': -2113931266},
 '_conn_attrs': {'_connector_name': 'mysql-connector-python',
  '_connector_license': 'GPL-2.0',
  '_connector_version': '8.0.32',
  '_source_host': 'Zen'},
 '_user': 'sensors',
 '_password': '##sensors##',
 '_password1': '',
 '_password2': '',
 '_password3': '',
 '_database': 'sensors',
 '_host': 'localhost',
 '_port': 3306,
 '_unix_socket': None,
 '_client_host': '',
 '_client_port': 0,
 '_ssl': {},
 '_ssl_disabled': False,


Relembrar que, no final de tudo o que queremos fazer com a BD, devemos libertar sempre a conexão

In [3]:
conn.close()

### Ficheiro de configuração e resolução de exceções

Está na hora de começarmos a pensar em como fazer código mais robusto e reutilizável:
1. Um primeiro passo é criar um ficheiro de configuração (i.e., `config.py`), onde definimos os parâmetros de acesso à BD:
   ``` python
   config = {
      'host': 'localhost',
      'user': 'sensors',
      'password': '##sensors##', #BAD! Deveria ser hash(pass+salt)
      'db': 'sensors'
   } # (idealmente deveria usar python-dotenv)
   ```
   * Isto permite que o código seja mais fácil de manter e de reutilizar, pois assim não é necessário alterá-lo quando queremos modificar os parâmetros de acesso à BD.

1. Um segundo passo é prever a resolução de exceções, para que o programa não termine quando ocorre um erro.




e depois fazer...

In [1]:
# Começamos por importar o ficheiro de configuração
import mysql.connector

from config import config

config

{'host': 'localhost',
 'user': 'sensors',
 'password': '##sensors##',
 'database': 'sensors'}

In [2]:
try:
    conn = mysql.connector.connect(**config)
except mysql.connector.Error as e:
    print("Ups! Ocorreu um erro na conexão!")
    print(dir(e))
    print(e.errno)
else:
    print("Sucesso!")
    conn.close()

Sucesso!


### Exercício
1. Experimente desligar o servidor e correr a linha acima: qual a mensagem de erro?
2. Mude o nome do utilizador no ficheiro de configuração (reinicie o kernel) e corra a linha acima: qual a mensagem de erro?
3. Re-implemente o código de modo a apresentar uma mensagem adequada quando falhar pelo servidor estar desligado e voltar a tentar mais 2 vezes a cada 5 segundos (vejam o pacote `time`, em particular o método `sleep()`)

In [6]:
import time

print("Olá!")
time.sleep(3)
print("Olá de novo!")

Olá!
Olá de novo!


## Operações de DDL: Criação de uma base de dados

Para a criação das tabelas e relacionamentos podemos construir um *statement* em **Structured Query Language (SQL)** ou, como alternativa, podemos usar ferramentas como sejam o MySQL Workbench, o phpMyAdmin, o SQlite Browser, o DataGrip, etc. 

Para quem ainda não conhece o SQL, recomenda-se o tutorial em https://www.w3schools.com/sql/sql_intro.asp

![Comandos SQL](https://www.w3schools.in/wp-content/uploads/2014/08/SQL-Commands-1-700x470.png)

*(figura retirada de https://www.w3schools.in/mysql/ddl-dml-dcl/)*

DDL is the short name for **Data Definition Language**, which deals with database schemas, descriptions, and how the data should reside in the database.

* `CREATE` - to create a database and its objects like (table, index, views, store procedure, function, and triggers)
* `ALTER` - alters the structure of the existing database
* `DROP` - delete objects from the database
* `TRUNCATE` - remove all records from a table, including all spaces allocated for the records are removed
* `COMMENT` - add comments to the data dictionary
* `RENAME` - rename an object

Consideremos o caso em que construímos o SQL statement:

Comecemos por criar uma base de dados no servidor de MySQL (façam sempre resolução de exceções...).

In [3]:
sql = """
USE `sensors`;

-- -----------------------------------------------------
-- Table `sensors`.`Location`
-- -----------------------------------------------------
CREATE TABLE
  IF NOT EXISTS `sensors`.`Location` (
    `idLocation` INT NOT NULL AUTO_INCREMENT,
    `name` VARCHAR(45) NOT NULL,
    `description` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`idLocation`),
    UNIQUE INDEX `name_UNIQUE` (`name` ASC)
  ) ENGINE = InnoDB;

-- -----------------------------------------------------
-- Table `sensors`.`Unit`
-- -----------------------------------------------------
CREATE TABLE
  IF NOT EXISTS `sensors`.`Unit` (
    `unit` VARCHAR(45) NOT NULL,
    `description` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`unit`)
  ) ENGINE = InnoDB;

-- -----------------------------------------------------
-- Table `sensors`.`Sensor`
-- -----------------------------------------------------
CREATE TABLE
  IF NOT EXISTS `sensors`.`Sensor` (
    `idSensor` INT NOT NULL AUTO_INCREMENT,
    `idLocation` INT NOT NULL,
    `name` VARCHAR(45) NOT NULL,
    `unit` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`idSensor`),
    INDEX `fk_Sensor_Location_idx` (`idLocation` ASC),
    INDEX `fk_Sensor_Units1_idx` (`unit` ASC),
    UNIQUE INDEX `uniq_loc_vs_sensor` (`idLocation` ASC, `name` ASC),
    CONSTRAINT `fk_Sensor_Location` FOREIGN KEY (`idLocation`) REFERENCES `sensors`.`Location` (`idLocation`) ON UPDATE cascade ON DELETE cascade,
    CONSTRAINT `fk_Sensor_Units1` FOREIGN KEY (`unit`) REFERENCES `sensors`.`Unit` (`unit`) ON UPDATE cascade ON DELETE cascade
  ) ENGINE = InnoDB;

-- -----------------------------------------------------
-- Table `sensors`.`Reading`
-- -----------------------------------------------------
CREATE TABLE
  IF NOT EXISTS `sensors`.`Reading` (
    `idReading` INT NOT NULL AUTO_INCREMENT,
    `idSensor` INT NOT NULL,
    `timestamp` TIMESTAMP NOT NULL,
    `value` FLOAT NOT NULL,
    PRIMARY KEY (`idReading`),
    INDEX `fk_Reading_Sensor1_idx` (`idSensor` ASC),
    CONSTRAINT `fk_Reading_Sensor1` FOREIGN KEY (`idSensor`) REFERENCES `sensors`.`Sensor` (`idSensor`) ON UPDATE cascade ON DELETE cascade
  ) ENGINE = InnoDB;

-- -----------------------------------------------------
-- Table `sensors`.`Alert`
-- -----------------------------------------------------
CREATE TABLE
  IF NOT EXISTS `sensors`.`Alert` (
    `idAlert` INT NOT NULL AUTO_INCREMENT,
    `idSensor` INT NOT NULL,
    `timestamp` TIMESTAMP NOT NULL,
    `description` VARCHAR(45) NOT NULL,
    `cleared` BIT NULL,
    PRIMARY KEY (`idAlert`),
    INDEX `fk_Alert_Sensor1_idx` (`idSensor` ASC),
    CONSTRAINT `fk_Alert_Sensor1` FOREIGN KEY (`idSensor`) REFERENCES `sensors`.`Sensor` (`idSensor`) ON UPDATE cascade ON DELETE cascade
  ) ENGINE = InnoDB;
"""

Depois de termos o SQL statement, fazemos:

In [4]:
# from config import config as conf
from config import config
import mysql.connector

try:
    # Abrimos a conexão com a base de dados
    conn = mysql.connector.connect(**config)  # dictionary unpacking
    
    # Criamos um cursor a partir da conexão
    cursor = conn.cursor()
    
    # Executamos o statement sql
    cursor.execute(sql, multi=True)
except mysql.connector.Error as err:
    print(err)
else:
    print("ok!")
    conn.close()

ok!


* O comando `cnx.cursor()` devolve um objeto da classe `MySQLCursor` que executa operações na BD através de instruções SQL.
* Por sua vez, os objetos do tipo `MySQLCursor` interagem com o servidor MySQL usando um objeto `MySQLConnection`.
* Para mais informações ver https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html

No phpMyAdmin devem ver algo como

![phpmyadmin_database_created.png](img/phpmyadmin_database_created.png)


## Operações CRUD (Create, Read, Update, Delete)
* Create, Read, Update, Delete são 4 operações fundamentais em qualquer modelo de armazenamento de dados.
* Em SQL, estas operações são implementadas através de comandos DML (**Data Manipulation Language**).
* Os comandos DML mais comuns são:
  * `INSERT` - para inserir dados numa tabela
  * `SELECT` - para selecionar dados de uma tabela
  * `UPDATE` - para atualizar dados numa tabela
  * `DELETE` - para apagar dados de uma tabela

### `INSERT` 

Aberta a conexão em MySQL

In [7]:
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

#### Localização
Uma boa estratégia é definir as variáveis no SQL usando parâmetros no estilo `%s` or `%(nome)s` e uma tupla com os dados

In [5]:
sql = """
    INSERT INTO Location (idLocation, name, description) 
    VALUES (DEFAULT, %s, %s)
"""
# e um tuplo com os dados
data = ("Prometheus Server II", "Prometheus Server @ lab. 163 / ISE /UAlg")

e agora inserir uma nova localização na base de dados e obter o id correspondente, guardado em `location_id` e que iremos usar à frente.

In [8]:
cursor.execute(sql, data) # ver https://pyformat.info/ – *basic formatting* e *named placeholders*
location_id = cursor.lastrowid
location_id

1

O método `.execute` garante que os dados são convertidos num formato compatível com os campos MySQL onde irão ser armazenados.

Importante, quando estamos a usar um sistema transacional, como o InnoDB, temos de efetuar o `COMMIT` depois de fazer um `INSERT`, `DELETE`, ou `UPDATE` (comandos que alterem tabelas).
   * ver (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-commit.html)

Nota: O InnoDB é o principal motor interno de armazenamento do MySQLServer (https://dev.mysql.com/doc/refman/8.0/en/innodb-introduction.html)

In [9]:
conn.commit()  # ou, para "undo", conn.rollback()

#### Tabela `Unit`
Inserir um novo registo na tabela `Unit`, substituíndo o anterior se tiver idêntica key (ver a documentação do comando [`REPLACE`](https://dev.mysql.com/doc/refman/5.7/en/replace.html) do SQL)

In [10]:
sql = """
    REPLACE INTO Unit (unit, description) 
    VALUES (%s, %s)
"""
data = ("percent", "percentage of usage")

cursor.execute(sql, data)  # statement + tupla com os dados
conn.commit()

#### Tabela `Sensor`
Inserir um novo sensor e obter o seu id, preparando o sql query

In [11]:
sql = """
    INSERT INTO Sensor (idSensor, idLocation, name, unit)
    VALUES (DEFAULT, %(idLocation)s, %(name)s, %(unit)s)
"""

Preparar os dados, agora com um dicionário que tem em conta `%(idLocation)s, %(name)s, %(unit)s`


In [12]:
data = {
    "idLocation": location_id,
    "name": "cpu_sensor_01",
    "unit": "percent",
}

Executar o statement

In [13]:
cursor.execute(sql, data)  # statement + dicionário com os dados
sensor_id = cursor.lastrowid
conn.commit()

In [14]:
sensor_id

1

#### Tabela `Readings`
E agora, vamos obter alguns dados e enviar para a base de dados.

Neste caso vamos usar a biblioteca `psutil` que permite obter informações sobre processos em execução e utilização do sistema (CPU, memória, discos, rede, sensores) em Python. (https://pypi.org/project/psutil/)

In [15]:
%pip install psutil

Note: you may need to restart the kernel to use updated packages.


In [17]:
import psutil

psutil.cpu_percent(interval=1)

1.4

Podemos também listar alguns sensores disponíveis no sistema

In [18]:
[o for o in dir(psutil) if o.startswith("cpu") or o.startswith("sensor")]

['cpu_count',
 'cpu_freq',
 'cpu_percent',
 'cpu_stats',
 'cpu_times',
 'cpu_times_percent',
 'sensors_battery']

In [19]:
import psutil

sql = """
    INSERT INTO Reading (idReading, idSensor, timestamp, value)     
    VALUES (DEFAULT, %(idSensor)s, DEFAULT, %(value)s)
"""

for _ in range(20):
    data = {
        "idSensor": sensor_id,
        "value": psutil.cpu_percent(interval=1),
    }

    cursor.execute(sql, data)
    conn.commit()
    print(".", end="")

....................

In [20]:
cursor.close()
conn.close()

## Selecionar dados em MySQL

O processo é simples dados os conhecimentos anteriores </div><i class="fa fa-lightbulb-o "></i>

In [21]:
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

Podemos usar o comando `SELECT` para selecionar dados de uma tabela. Neste caso vamos procurar pelo local que tem uma `description` em que contenha "163".

In [22]:
sql = """
    SELECT idLocation, name, description 
    FROM Location 
    WHERE description LIKE "%163%"
"""

cursor.execute(sql)

In [23]:
cursor

Podemos percorrer os dados obtidos pelo `cursor` usando, por exemplo, um ciclo `for`

In [24]:
for (idLocation, name, description) in cursor:
    print(f"id: {idLocation}\n\t name: {name} \n\t description: {description}")

id: 1
	 name: Prometheus Server II 
	 description: Prometheus Server @ lab. 163 / ISE /UAlg


Outros exemplos

In [25]:
sql = """
    SELECT idReading, idSensor, timestamp, value 
    FROM Reading 
    WHERE value BETWEEN %s and %s
"""
data = (1, 2)

cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value) in cursor:
    print(
        f"idReading: {idReading}\n\t idSensor: {idSensor} \n\t time: {timestamp} \n\t value: {value}"
    )

idReading: 1
	 idSensor: 1 
	 time: 2024-03-22 15:23:03 
	 value: 1.6
idReading: 3
	 idSensor: 1 
	 time: 2024-03-22 15:23:05 
	 value: 1.4
idReading: 4
	 idSensor: 1 
	 time: 2024-03-22 15:23:06 
	 value: 2.0
idReading: 5
	 idSensor: 1 
	 time: 2024-03-22 15:23:07 
	 value: 1.2
idReading: 7
	 idSensor: 1 
	 time: 2024-03-22 15:23:09 
	 value: 1.6
idReading: 15
	 idSensor: 1 
	 time: 2024-03-22 15:23:17 
	 value: 2.0
idReading: 17
	 idSensor: 1 
	 time: 2024-03-22 15:23:19 
	 value: 1.2
idReading: 20
	 idSensor: 1 
	 time: 2024-03-22 15:23:22 
	 value: 1.2


In [26]:
sql = """
    SELECT *
    FROM Location AS L
    INNER JOIN Sensor AS S ON L.idLocation = S.idLocation
    INNER JOIN Unit AS U ON S.unit = U.unit
    INNER JOIN Reading AS R ON S.idSensor = R.idSensor
    WHERE value BETWEEN %(low)s AND %(high)s
    ORDER BY value
"""

data = {
    "low": 1,
    "high": 2,
}

cursor.execute(sql, data)

* Podemos obter os nomes e outros dados das colunas (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-description.html)
   * Formato: (name, type_code, display_size, internal_size, precision, scale, null_ok, column_flags)


In [27]:
cursor.description

[('idLocation', 3, None, None, None, None, 0, 32769, 63),
 ('name', 253, None, None, None, None, 0, 4097, 45),
 ('description', 253, None, None, None, None, 0, 4097, 45),
 ('idSensor', 3, None, None, None, None, 0, 32769, 63),
 ('idLocation', 3, None, None, None, None, 0, 36865, 63),
 ('name', 253, None, None, None, None, 0, 4097, 45),
 ('unit', 253, None, None, None, None, 0, 4097, 45),
 ('unit', 253, None, None, None, None, 0, 4097, 45),
 ('description', 253, None, None, None, None, 0, 4097, 45),
 ('idReading', 3, None, None, None, None, 0, 32769, 63),
 ('idSensor', 3, None, None, None, None, 0, 36865, 63),
 ('timestamp', 7, None, None, None, None, 0, 161, 63),
 ('value', 4, None, None, None, None, 0, 36865, 63)]

In [30]:
from mysql.connector import FieldType

for i in range(len(cursor.description)):
    print(f"Column {i + 1}:")
    desc = cursor.description[i]
    print(f"  column_name = {desc[0]}")
    print(f"  type = {desc[1]} ({FieldType.get_info(desc[1])})")
    print(f"  null_ok = {desc[6]}")

Column 1:
  column_name = idLocation
  type = 3 (LONG)
  null_ok = 0
Column 2:
  column_name = name
  type = 253 (VAR_STRING)
  null_ok = 0
Column 3:
  column_name = description
  type = 253 (VAR_STRING)
  null_ok = 0
Column 4:
  column_name = idSensor
  type = 3 (LONG)
  null_ok = 0
Column 5:
  column_name = idLocation
  type = 3 (LONG)
  null_ok = 0
Column 6:
  column_name = name
  type = 253 (VAR_STRING)
  null_ok = 0
Column 7:
  column_name = unit
  type = 253 (VAR_STRING)
  null_ok = 0
Column 8:
  column_name = unit
  type = 253 (VAR_STRING)
  null_ok = 0
Column 9:
  column_name = description
  type = 253 (VAR_STRING)
  null_ok = 0
Column 10:
  column_name = idReading
  type = 3 (LONG)
  null_ok = 0
Column 11:
  column_name = idSensor
  type = 3 (LONG)
  null_ok = 0
Column 12:
  column_name = timestamp
  type = 7 (TIMESTAMP)
  null_ok = 0
Column 13:
  column_name = value
  type = 4 (FLOAT)
  null_ok = 0


In [31]:
lista_de_colunas = [t[0] for t in cursor.description]
lista_de_colunas

['idLocation',
 'name',
 'description',
 'idSensor',
 'idLocation',
 'name',
 'unit',
 'unit',
 'description',
 'idReading',
 'idSensor',
 'timestamp',
 'value']

In [32]:
for linha in cursor:
    print(
        "\t".join(
            [f"| {coluna}: {valor}" for coluna, valor in zip(lista_de_colunas, linha)]
        )
    )

| idLocation: 1	| name: Prometheus Server II	| description: Prometheus Server @ lab. 163 / ISE /UAlg	| idSensor: 1	| idLocation: 1	| name: cpu_sensor_01	| unit: percent	| unit: percent	| description: percentage of usage	| idReading: 8	| idSensor: 1	| timestamp: 2023-03-19 23:08:37	| value: 1.0
| idLocation: 1	| name: Prometheus Server II	| description: Prometheus Server @ lab. 163 / ISE /UAlg	| idSensor: 1	| idLocation: 1	| name: cpu_sensor_01	| unit: percent	| unit: percent	| description: percentage of usage	| idReading: 1	| idSensor: 1	| timestamp: 2023-03-19 23:08:30	| value: 1.1
| idLocation: 1	| name: Prometheus Server II	| description: Prometheus Server @ lab. 163 / ISE /UAlg	| idSensor: 1	| idLocation: 1	| name: cpu_sensor_01	| unit: percent	| unit: percent	| description: percentage of usage	| idReading: 5	| idSensor: 1	| timestamp: 2023-03-19 23:08:34	| value: 1.2
| idLocation: 1	| name: Prometheus Server II	| description: Prometheus Server @ lab. 163 / ISE /UAlg	| idSensor: 1	

### o comando `fetchall`
Com o comando `fetchall` podemos obter todos os resultados de uma única vez como uma lista de tuplas

In [33]:
# É necessário voltar a correr o SELECT pois o cursor é um iterável e foi esvaziado
cursor.execute(sql, data)

cursor.fetchall()

[(1,
  'Prometheus Server II',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  8,
  1,
  datetime.datetime(2023, 3, 19, 23, 8, 37),
  1.0),
 (1,
  'Prometheus Server II',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  1,
  1,
  datetime.datetime(2023, 3, 19, 23, 8, 30),
  1.1),
 (1,
  'Prometheus Server II',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  5,
  1,
  datetime.datetime(2023, 3, 19, 23, 8, 34),
  1.2),
 (1,
  'Prometheus Server II',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  2,
  1,
  datetime.datetime(2023, 3, 19, 23, 8, 31),
  1.3),
 (1,
  'Prometheus Server II',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
 

Podemos também converter para um dicionário mas no **nosso exemplo** (dado o `*` no `SELECT`) **NÃO é boa ideia** pois há colunas que têm o mesmo nome (e.g., `nome`), pelo que se perdem colunas ao passar para um dicionário.

In [34]:
# É necessário voltar a correr o SELECT pois o cursor foi esvaziado
cursor.execute(sql, data)

for linha in cursor:
    print({coluna: valor for valor, coluna in zip(linha, lista_de_colunas)})
    # Só ficou um 'name'!

{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 8, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 37), 'value': 1.0}
{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 1, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 30), 'value': 1.1}
{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 5, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 34), 'value': 1.2}
{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 2, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 31), 'value': 1.3}
{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 6, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 35), 'value': 

In [35]:
cursor.close()
conn.close()

### Dados na forma de dicionários

Quando se cria o cursor com o parâmetro `dictionary=True` ao iterar sobre os resultados estes vêm na forma de dicionários

In [36]:
conn = mysql.connector.connect(**config)
cursor = conn.cursor(dictionary=True)

sql = """
    SELECT idReading, idSensor, timestamp, value 
    FROM Reading 
    WHERE value BETWEEN %s and %s
"""
data = (5, 50)

cursor.execute(sql, data)

for linha in cursor:
    print(linha)

{'idReading': 17, 'idSensor': 1, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 46), 'value': 6.8}
{'idReading': 18, 'idSensor': 1, 'timestamp': datetime.datetime(2023, 3, 19, 23, 8, 47), 'value': 7.8}


In [37]:
cursor.close()
conn.close()

### EXERCÍCIO
Utilize os pacotes `time` e `psutil` para calcular a memoria virtual livre a cada segundo durante 30 segundos, guardando na base de dados. Use
```
...
x = psutil.virtual_memory()
x.free
...
```